In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import math

# Bandit Problom

##  Problom description
0. 카지노에 방문했다고 가정해보자.
1. casino에 10개의 bandit(slot) machine이 있다.
2. 각각의 bandit의 당첨 확률이 다르다. (서로다른 probabilistic distribution을 갖는다)
3. 한번에 한개 bandit의 arm을 당길 수 있고, 결과에 따라 reward를 받는다.
4. 24시간 동안 플레이 할 수 있고 최대 10000번의 게임을 수행할 수 있다.
5. 자, 이제 하루동안 최대의 수익(reward)을 얻고 싶다. 어떤 전략(policy)으로 bandit을 선택하고 당겨야 할까??

### exploitation & exploration의 개념
exploitation
* 기존 관측된 사실과 경험을 토대로 가장 좋은 bandit을 선택함

exploration
* 새로운 관측 사실과 경험을 쌓기 위해 새로운 bandit을 선택함

## Policy
1. Random Selection
    - 가장 단순하게 골고루 랜덤하게 선택한다.
2. K exploration & N-K exploitation (Bucket Test)
    - K번을 랜덤하게 선택하고 관측 사실과 경험을 쌓음 (exploration)
    - 나머지 N-K번은 위 가장 성적이 좋은 bandit 선택함 (exploitaion)
3. Epsilon-Greedy
    - epsilon을 선택하고 랜덤 확률을 추출 함
    - 추출 값이 epsilon 보다 작으면 exploration함 (epsilon probability)
    - 추출 값이 epsilon 보다 크면 exploitaion함 (1-epsilon probability)
4. UCB(Upper Confidence Bound)
    - $ucb_{i} = \mu_{i} + \sqrt\frac{2*\ln t}{n_{i}}$
    - $i = \arg \max_{i}(ucb_{i})$
    - i: bandit, t: 현재 게임 횟수, $\mu_i$: t시점의 평균, $n_i$: bandit i의 게임 횟수 
    - 위 식에 의해 각각의 bandit i의 ucb를 계산하고 max i를 선택함
    - 위 ucb 식에서 second term은 다양하게 정의 될 수 있다.
        - 위 식은 가장 단순한 UCB1이고, UCB2, UCB-Tuned, MOSS, KL-UCB, Bayes-UCB 등이 있다.
        - 참조: https://arxiv.org/pdf/1510.00757.pdf
5. Thomson Sampling
    

# Toy DataSet

위 bandit 문제를 광고 도메인에서 생각해보자.
1. 하나의 광고주에서 10개의 서로 다른 캠페인을 만들었다.
2. 광고 한번 노출하는데 -10원, 광고를 클릭하면 +100원 이라고 해보자.
3. Independent한 10000명의 user에게 노출할 수 있는 예산(10만원)이 있다.
    - bandit에서 첫번째 시도의 결과과 두번째 시도의 결과에 영향을 주지 않는다.
    - 사전 유저에 대한 성향을 알 수 없다.
    - 광고에서 한 유저에게 하나의 광고를 한번 노출해야 하고, A유저가 B유저에 영향을 미치지 않는다.
4. 전지전능하여 10000명의 user에 대하여 어떤 캠페인에 클릭할지를 미리 알고 있다.
    - 아래 데이터의 행은 유저를, 열은 광고를 나타내고 광고를 클릭할 것이면 1 아니면 0으로 표현한다.
5. 각 광고의 click수와 ctr을 알 수 있다.
    - Ad 5가 클릭 2695개로 가장 좋고, Ad 6이 click 126개로 가장 나쁘다.
    - 즉, 10000번을 처음부터 Ad 5 광고에 몰빵하는 것이 최대 수익을 낼 수 있다.

In [3]:
dataset = pd.read_csv('Ads_Optimisation.csv')
print(dataset.head())
dataset.agg(['sum', 'mean']).rename(index={'sum':'click', 'mean':'ctr'})

   Ad 1  Ad 2  Ad 3  Ad 4  Ad 5  Ad 6  Ad 7  Ad 8  Ad 9  Ad 10
0     1     0     0     0     1     0     0     0     1      0
1     0     0     0     0     0     0     0     0     1      0
2     0     0     0     0     0     0     0     0     0      0
3     0     1     0     0     0     0     0     1     0      0
4     0     0     0     0     0     0     0     0     0      0


,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
click,1703.0000,1295.0000,728.0000,1196.0000,2695.0000,126.0000,1112.0000,2091.0000,952.0000,489.0000
ctr,0.1703,0.1295,0.0728,0.1196,0.2695,0.0126,0.1112,0.2091,0.0952,0.0489


# 1. Random Selection

In [4]:
random.seed(1050)
N=10000  ## users
d=10     ## ads
ads_selected = []
rewards = []
for n in range(N):
    ad = random.randrange(d)         # 0~9까지 랜덤하게 광고를 선택한다.
    ads_selected.append(ad)          # 추출된 광고를 노출한다.
    reward = dataset.values[n, ad]   # 해당 유저가 클릭하면 1 아니면 0을 준다.
    rewards.append(reward)           # 각 trial에 수익을 기록한다.

In [5]:
def result(ads_selected, rewards, dataset=dataset):
    ## N impression에 대한 reward 값을 data frame으로 만든다.
    rs = pd.DataFrame({'ad': ads_selected, 'reward': rewards})
    ## ad 를 key 그룹바이로 size는 노출수가 되고 sum은 click수가 된다. 노출과 클릭으로 부터 ctr을 계산한다.
    rs_agg = rs.groupby('ad').agg(['size','sum'])
    rs_agg.columns = rs_agg.columns.droplevel(0)
    rs_agg = rs_agg.rename(columns={'size': 'impression', 'sum': 'click'})
    rs_agg['ctr'] = rs_agg['click']/rs_agg['impression']
    ## true ctr 값과 비교해 본다.
    rs_agg['true_ctr'] = dataset.agg('mean').values
    return(rs_agg)

랜덤하게 광고를 선택할 경우 10000개의 노출 중 **1257**개가 클릭되었다. <br>
노출당 -10원 클릭당 +100원이므로 매출은 125,700원이고 순이익은 25,700원이 된다. <br>
ctr이 true_ctr과 유사하게 나타난다. 

In [6]:
rs_agg = result(ads_selected, rewards)
print('total clicks:', rs_agg.click.sum())
print('total revenue:', rs_agg.click.sum() * 100, '원')
rs_agg

total clicks: 1257
total revenue: 125700 원


,impression,click,ctr,true_ctr
ad,,,,
0,981,159,0.162080,0.1703
1,1005,135,0.134328,0.1295
2,997,66,0.066199,0.0728
3,1012,124,0.122530,0.1196
4,1018,278,0.273084,0.2695
5,1005,17,0.016915,0.0126
6,977,117,0.119754,0.1112
7,1048,229,0.218511,0.2091
8,952,94,0.098739,0.0952


# 2. K exploration & N-K exploitation

In [7]:
random.seed(1050)
N=10000  ## users
d=10     ## ads
K=1000   ## exploration 수
bucket = np.repeat(range(d), K/d) ## K유저에 bucket 할당
random.shuffle(bucket)            ## bucket random shuffle

def K_exploration(dataset, N, K, bucket):
    ads_selected = []
    rewards = []

    for n in range(K):
        ad = bucket[n]                   # 해당 유저가 할당된 bucket의 광고를 선택한다.
        ads_selected.append(ad)          # 선택된 광고를 노출한다.
        reward = dataset.values[n, ad]   # 해당 유저가 클릭하면 1 아니면 0을 준다.
        rewards.append(reward)           # 수익을 기록한다.

    # K exploration 동안 performance(ctr)이 가장 좋은 광고를 선택한다.
    ad_reward = {i: 0 for i in range(10)}
    for ad, re in zip(ads_selected, rewards):
        ad_reward[ad] += re
    max_ctr_ad = max(ad_reward, key=lambda key: ad_reward[key])
    # N-K 유저에게 가장 성능이 좋은 광고를 노출 시킨다.
    ads_selected.extend([max_ctr_ad]*(N-K))
    rewards.extend(dataset.values[K:, max_ctr_ad])
    return max_ctr_ad, ads_selected, rewards

max_ctr_ad, ads_selected, rewards = K_exploration(dataset, N, K, bucket)

처음 1000명의 exploration 동안 Ad 5가 가장 좋은 성능을 보이는 광고로 선택되었고, <br>
나머지 9000명에게 Ad 5를 exploitation 한 결과 총 **2549**개가 클릭되었다. <br>
노출당 -10원 클릭당 +100원이므로 매출은 254,900원이고 순이익은 154,900원이 된다. <br>
ctr이 true_ctr과 유사하게 나타난다. 

In [8]:
print('max ad:', 'Ad', max_ctr_ad+1)
rs_agg = result(ads_selected, rewards)
print('total clicks:', rs_agg.click.sum())
print('total revenue:', rs_agg.click.sum() * 100, '원')
rs_agg

max ad: Ad 5
total clicks: 2549
total revenue: 254900 원


,impression,click,ctr,true_ctr
ad,,,,
0,100,19,0.19000,0.1703
1,100,13,0.13000,0.1295
2,100,8,0.08000,0.0728
3,100,10,0.10000,0.1196
4,9100,2460,0.27033,0.2695
5,100,2,0.02000,0.0126
6,100,11,0.11000,0.1112
7,100,14,0.14000,0.2091
8,100,9,0.09000,0.0952


위 case같은 경우는 1000번의 exploration에서 성능이 가장 좋은 광고를 찾았기 때문에 나머지 exploitation 동안 수익을 최대화 할 수 있었다. <br>
만약 아래 예와 같이 1000번의 exploration 동안 성능이 가장 좋은 광고를 찾지 못한다면, 최대의 수익을 얻지 못할 것이다. <br>
K exploration & N-K exploitation 방법은 exploration 수(K)의 선택에 따라 결과과 크게 달라질 수 있는데, 적절한 K를 선택해야하는 문제가 있다.

In [9]:
## 초반 1000개 까지는 Ad 8의 성적이 우수하도록 만들어 본다.
np.random.seed(1234)
idx8_1 = np.random.choice(np.where(dataset[:1000]['Ad 8'] == 0)[0], 100, replace=False)
idx8_2 = np.random.choice(np.where(dataset[1000:]['Ad 8'] == 1)[0], 100, replace=False) + 1000
idx5_1 = np.random.choice(np.where(dataset[:1000]['Ad 5'] == 1)[0], 100, replace=False)
idx5_2 = np.random.choice(np.where(dataset[1000:]['Ad 5'] == 0)[0], 100, replace=False) + 1000

dataset['Ad 8'][idx8_1] = 1
dataset['Ad 8'][idx8_2] = 0
dataset['Ad 5'][idx5_1] = 0
dataset['Ad 5'][idx5_2] = 1

위에서 초반에 Ad 8의 성적을 Ad 5보다 좋게 만든 다면, 1000번의 exploration 동한 max ctr ad가 Ad 8이 선택되고 성과가 떨어지게 된다. <br>
이와 같이 확률분포가 일정하여 초반에 정확히 성능이 좋은 광고를 선택할 수 있으면 좋은 성과를 가질 수 있지만, 확률분포가 일정하지 않고 변한다면, K exploration 동안 적절한 광고를 선택하지 못하고 성과를 최대화 하지 못할 것이다.

In [10]:
max_ctr_ad, ads_selected, rewards = K_exploration(dataset, N, K, bucket)
print('max ad:', 'Ad', max_ctr_ad+1)
rs_agg = result(ads_selected, rewards)
print('total clicks:', rs_agg.click.sum())
print('total revenue:', rs_agg.click.sum() * 100, '원')
rs_agg

max ad: Ad 8
total clicks: 1902
total revenue: 190200 원


,impression,click,ctr,true_ctr
ad,,,,
0,100,19,0.190000,0.1703
1,100,13,0.130000,0.1295
2,100,8,0.080000,0.0728
3,100,10,0.100000,0.1196
4,100,20,0.200000,0.2695
5,100,2,0.020000,0.0126
6,100,11,0.110000,0.1112
7,9100,1807,0.198571,0.2091
8,100,9,0.090000,0.0952


In [11]:
## data rollback
dataset = pd.read_csv('Ads_Optimisation.csv')

# 3. Epsilon-Greedy

In [12]:
random.seed(1050)
N=10000  ## users
d=10     ## ads
eps=0.2  
ads_selected = []
rewards = []
ad_imp = {i: 1e-8 for i in range(10)} ## 광고별 impression 누적 dic
ad_reward = {i: 0 for i in range(10)} ## 광고별 reward 누적 dic


for n in range(N):
    p = random.random()  ## 랜덤 확률 추출
    ## 누적 reward와 impression dic으로 부터 max ctr ad 선택
    ads_ctr = {x: ad_reward.get(x) / ad_imp.get(x) for x in ad_imp}
    max_ctr_ad = max(ads_ctr, key=lambda key: ads_ctr[key])

    ## 랜덤 추출한 확률 p가 epsilon보다 작으면 max ctr ad가 아닌 광고중 랜덤하게 선택한다.
    ## 그렇지 않으면, max ctr ad를 선택한다.
    if(p < eps):
        choice = True
        while choice:
            ad = random.randrange(d)
            choice = ad==max_ctr_ad
    else:
        ad = max_ctr_ad
    ## 결과를 기록 및 누적한다.
    ads_selected.append(ad)          
    reward = dataset.values[n, ad]   
    rewards.append(reward)  
    ad_imp[ad] +=1
    ad_reward[ad] += reward

epsilon 값을 높이면 random selection 방법과 유사해진다. <br>
위 실행에서는 0.2로 설정 했고, 결과 총 클릭 **2352**개를 얻을 수 있다. <br>
ctr이 true ctr과 비교해서 약간 차이를 보인다.

In [13]:
rs_agg = result(ads_selected, rewards)
print('total clicks:', rs_agg.click.sum())
print('total revenue:', rs_agg.click.sum() * 100, '원')
rs_agg

total clicks: 2352
total revenue: 235200 원


,impression,click,ctr,true_ctr
ad,,,,
0,275,54,0.196364,0.1703
1,462,65,0.140693,0.1295
2,252,27,0.107143,0.0728
3,234,23,0.098291,0.1196
4,7631,2069,0.271131,0.2695
5,200,0,0.000000,0.0126
6,188,24,0.127660,0.1112
7,278,50,0.179856,0.2091
8,237,25,0.105485,0.0952


# 4. UCB(Upper Confidence Bound)

In [20]:
random.seed(1050)
N=10000  ## users
d=10     ## ads
ads_selected = []
rewards = []
ad_imp = {i: 1e-8 for i in range(10)} ## 광고별 impression 누적 dic
ad_reward = {i: 0 for i in range(10)} ## 광고별 reward 누적 dic

for n in range(N):
    ## 매 시행마다 광고별 impression과 reward로 부터 ucb를 계산한다.
    ad_ucb=[]
    for i in range(d):
        average_reward = ad_reward[i] / ad_imp[i]
        delta_i = math.sqrt(2 * math.log(n+1) / ad_imp[i])
        upper_bound = average_reward + delta_i
        ad_ucb.append(upper_bound)
    ## max ucb 광고를 선택한다.
    ad = np.argmax(ad_ucb)
    ## 결과를 기록 및 누적한다.
    ads_selected.append(ad)
    ad_imp[ad] += 1
    reward = dataset.values[n, ad]
    rewards.append(reward)  
    ad_reward[ad] += reward

초반에는 여러 광고들이 다양하게 선택되어지며 탐색을 하고, 중.후반으로 갈 수 록 성능이 좋은 광고는 더 선택되고 안좋은 광고는 선택되는 비율이 적어진다.

In [26]:
print(np.unique(ads_selected[:1000], return_counts=True))
print(np.unique(ads_selected[5000:6000], return_counts=True))
print(np.unique(ads_selected[9000:10000], return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([115,  80,  77,  93, 153,  59,  80, 192,  96,  55]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 59,  44,  27,  24, 601,  14,  56, 110,  50,  15]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([106,   7,  26,  34, 771,   4,   7,  34,   6,   5]))


결과 총 클릭수는 **2125**개로 순이익 112500원을 얻을 수 있다. <br>
ctr은 true ctr과 유사하게 나타난다.

In [21]:
rs_agg = result(ads_selected, rewards)
print('total clicks:', rs_agg.click.sum())
print('total revenue:', rs_agg.click.sum() * 100, '원')
rs_agg

total clicks: 2125
total revenue: 212500 원


,impression,click,ctr,true_ctr
ad,,,,
0,947,176,0.185850,0.1703
1,417,48,0.115108,0.1295
2,338,31,0.091716,0.0728
3,380,40,0.105263,0.1196
4,5630,1519,0.269805,0.2695
5,180,1,0.005556,0.0126
6,435,52,0.119540,0.1112
7,1106,217,0.196203,0.2091
8,352,34,0.096591,0.0952


# 5. Thomson Sampling

In [37]:
np.random.seed(1050)
N=10000   ## users
d=10      ## ads
ads_selected = []
rewards = []
ck_success = {i: 0 for i in range(10)}  ## 광고별 노출 후 클릭한 수 dic
ck_failure = {i: 0 for i in range(10)}  ## 광고별 노출 후 클릭하지 않은 수 dic

for n in range(N):
    ## 매 시행마다 광고별 성공,실패 수로부터 beta 확률을 추출한다.
    beta_prob = []
    for i in range(d):
        prob = np.random.beta(ck_success[i]+1, ck_failure[i]+1, size=1)
        beta_prob.extend(prob)
    ## beta 확률이 가장 높은 광고를 선택한다.
    ad = np.argmax(beta_prob)
    ## 결과를 기록 및 누적한다.
    ads_selected.append(ad)
    reward = dataset.values[n, ad]
    rewards.append(reward)  
    if reward==1:
        ck_success[ad] += 1
    else:
        ck_failure[ad] += 1 

ubc 보다 초반에 성능이 좋은 광고로 더 빠르게 수렴한다. 중.후반에는 거의 Ad 5만 선택되어 진다.


In [39]:
print(np.unique(ads_selected[:1000], return_counts=True))
print(np.unique(ads_selected[5000:6000], return_counts=True))
print(np.unique(ads_selected[9000:10000], return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([155,  29,  20,  36, 436,  19,  44, 208,  22,  31]))
(array([0, 1, 3, 4, 6, 7]), array([  7,   3,   3, 961,   2,  24]))
(array([0, 1, 2, 3, 4, 7, 8]), array([  8,   2,   1,   1, 980,   7,   1]))


결과 총 클릭수는 2564개로 순이익 156400원을 얻을 수 있다. <br>
성능이 떨어져 초반부터 충분한 선택을 받지 못한 광고들의 ctr은 true_ctr과 차이가 발생하지만, 어느정도 선택된 광고들의 ctr은 true값과 유사해지는 것으로 보인다.

In [38]:
rs_agg = result(ads_selected, rewards)
print('total clicks:', rs_agg.click.sum())
print('total revenue:', rs_agg.click.sum() * 100, '원')
rs_agg

total clicks: 2564
total revenue: 256400 원


,impression,click,ctr,true_ctr
ad,,,,
0,252,50,0.198413,0.1703
1,133,20,0.150376,0.1295
2,23,0,0.000000,0.0728
3,64,7,0.109375,0.1196
4,8734,2339,0.267804,0.2695
5,24,0,0.000000,0.0126
6,82,10,0.121951,0.1112
7,614,135,0.219870,0.2091
8,33,1,0.030303,0.0952


# reference
각 알고리즘에 대한 자세한 수식 및 내용은 아래 참조.
* https://www.analyticsvidhya.com/blog/2018/09/reinforcement-multi-armed-bandit-scratch-python/
* http://sanghyukchun.github.io/96/
* https://brunch.co.kr/@chris-song/66